In [3]:
import os
import pickle
import pandas as pd
import numpy as np
import xgboost as xgb
from calculate_confusion_matrix import calculate_confusion_matrix
import time
from copy import deepcopy
from utils import one_hot_encoder
from keras.models import Sequential
from keras.layers import Dense, Activation

nn = Sequential()
nn.add(Dense(32, input_shape=(500,)))

# from sklearn.preprocessing import OneHotEncoder
# from sklearn import preprocessing

save_results = True
do_stratify = False

ft_dir = 'features_long/'

# list feature files
files = os.listdir(ft_dir)

# reading top locations
with open('top_locations.dat') as f:
    location_top = pickle.load(f)
f.close()

# reading top reasons
with open('top_reasons.dat') as f:
    reason_top = pickle.load(f)
f.close()
    
feature_all = []
target_all = []
for filename in files:
    with open(ft_dir+filename) as f:  
        feature, target = pickle.load(f)

        # only keeping top locations
        ind = np.array([], int)
        for (i,loc) in enumerate(target['location']):
            if loc in location_top:
                ind = np.append(ind, i)
        feature = feature.loc[ind,:]
        target = target.loc[ind]
        feature = feature.reset_index(drop=True)
        target = target.reset_index(drop=True)
        
        # only keeping top reasons
        ind = np.array([], int)
        for (i,r) in enumerate(target['reason']):
            if r in reason_top:
                ind = np.append(ind, i)
        feature = feature.loc[ind,:]
        target = target.loc[ind]
        feature = feature.reset_index(drop=True)
        target = target.reset_index(drop=True)

        feature_all.append(feature)
        target_all.append(target)
        
    f.close()

confs = []
aucs = []
labels = []
confs_fsq = []
aucs_fsq = []

for i in range(1):#range(len(feature_all)):
    
    print '------------------'
    print i
    if i==6:
        print 'subject skipped because of lack of data'
        continue
    
    # training set
    j_range = range(len(feature_all))
    j_range.pop(i)
    
    x_train = pd.concat([feature_all[j] for j in j_range], axis=0)
    y_train = pd.concat([target_all[j]['location'] for j in j_range], axis=0)
#     y_train = pd.concat([target_all[j]['reason'] for j in j_range], axis=0)
    
    x_train = x_train.reset_index(drop=True)
    y_train = y_train.reset_index(drop=True)
    
#     if do_stratify:
#         x_train, y_train = stratify(x_train,y_train)
    
    # test set
    x_test = feature_all[i]
    y_test = target_all[i]['location']
#     y_test = target_all[i]['reason']
    
    # remove foursquare data
#     x_train = x_train.drop(['fsq 0','fsq 1','fsq 2','fsq 3','fsq 4','fsq 5','fsq 6','fsq 7'],axis=1)
#     x_test = x_test.drop(['fsq 0','fsq 1','fsq 2','fsq 3','fsq 4','fsq 5','fsq 6','fsq 7'],axis=1)
    
    # train (layer 1)
    #eta_list = np.array([0.05]*200+[0.02]*200+[0.01]*200)
    gbm = xgb.XGBClassifier(max_depth=3, n_estimators=100, learning_rate=0.01, nthread=12, subsample=1,\
                               max_delta_step=0).fit(x_train, y_train)
    
    # train performance
#     y_pred = gbm.predict(x_train)
#     conf_train, roc_auc_train = calculate_confusion_matrix(y_pred, y_train)

    # test (layer 1)
    y_pred = gbm.predict(x_test)
    
    # test performance
    conf, roc_auc = calculate_confusion_matrix(y_pred, y_test)
    
    # foursquare performance
    #conf_fsq, roc_auc_fsq = calculate_confusion_matrix(state_fsq_all[i], y_test)
    
    labels.append(np.unique(y_test))
    confs.append(conf)
    aucs.append(roc_auc)
    #confs_fsq.append(conf_fsq)
    #aucs_fsq.append(roc_auc_fsq)

#     print 'train'
#     print np.unique(y_train)
#     #print conf
#     print np.nanmean(roc_auc_train)

#     print 'test'

    print np.unique(y_test)
    #print conf
    print roc_auc
    #print 'foursquare:'
    #print roc_auc_fsq
    
# saving the results
if save_results:
    with open('auc_nn_location_sensor_fsq.dat','w') as f:
        #pickle.dump([aucs, confs, labels, aucs_fsq, confs_fsq], f)
        pickle.dump([aucs, confs, labels], f)
    f.close()



------------------
0


KeyboardInterrupt: 

In [4]:
feature

,lgt mean,lgt std,lgt off,lgt zcrossing,lgt skew,lgt kurt,aud mean,aud std,aud skew,aud kurt,...,fsq 1,fsq 2,fsq 3,fsq 4,fsq 5,fsq 6,fsq 7,fsq distance,LT frequency,LT interval mean
0,NaN,NaN,NaN,NaN,NaN,NaN,0.005274,0.014165,3.220672e+00,9.295789,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,215.0,0.619048,1.627451
1,51.254308,43.771248,0.343609,-0.000066,0.251401,-0.111971,0.000872,0.002658,4.913214e+00,25.526247,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,216.0,0.619048,1.627451
2,0.171079,0.397624,0.835031,-0.004073,2.322822,6.197177,0.002364,0.000311,0.000000e+00,-2.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,17.0,0.011905,NaN
3,31.062284,19.716329,0.094603,0.000000,-0.190862,-1.220656,0.005512,0.005620,2.075440e+00,5.291514,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,357.0,0.059524,16.250000
4,50.594213,44.197526,0.329419,0.000000,0.493056,1.511310,0.001617,0.003146,3.085951e+00,10.868013,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,210.0,0.583333,1.729167
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,244.0,0.000000,NaN
6,64.841638,21.798136,0.060269,0.000059,-2.165114,3.912102,0.005301,0.021423,6.363223e+00,41.201717,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,218.0,0.630952,1.596154
7,0.001020,0.031919,0.998980,0.000000,31.265013,975.501021,0.012441,0.002690,3.629060e-01,-1.016087,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,244.0,0.011905,NaN
8,50.417415,41.118422,0.344506,-0.000138,-0.105525,-1.172926,0.001563,0.004215,4.762809e+00,23.976891,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,212.0,0.559524,1.804348
9,0.000453,0.021286,0.999547,0.000000,46.936132,2201.000454,0.002157,0.001244,7.818101e-01,-0.564922,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0,0.011905,NaN
